# Tema 6: RAG (Retrieval-Augmented Generation)

## Ejercicio A1 - Pipeline RAG y puntos de fallo

Explica el pipeline RAG completo (ingestión → chunking → embeddings → vector store → retrieval → prompting → generación) e identifica al menos 4 puntos donde el sistema puede fallar (p.ej., chunking mal calibrado, retrieval irrelevante, etc.). Para cada punto de fallo, propone una mitigación concreta.

### Pipeline completo

Ingestión de documentos → limpieza → chunking → embeddings → indexado en vector store → retrieval (top-k/MMR) → construcción de prompt con contexto → generación → post-procesado (citas/guardrails).

### Puntos de fallo típicos y mitigaciones

| # | Punto de fallo | Mitigación |
|---|---|---|
| 1 | **Chunking demasiado grande/pequeño** | Ajustar `chunk_size`/`overlap`. Usar splitters por estructura (párrafos, secciones). |
| 2 | **Embeddings inadecuados** (dominio/idioma) | Usar embeddings específicos de dominio o multilingües; normalización de vectores. |
| 3 | **Retrieval con ruido** | MMR para diversificar, filtros por metadatos, re-ranking, query rewriting. |
| 4 | **Context window saturada** | Compresión de contexto, top-k adaptativo, reducción por scoring. |
| 5 | **Generación no fiel** (alucinaciones) | Prompt con "use only context", forzar citas; verificación de groundedness. |

## Ejercicio A2 - RAG vs Fine-Tuning vs Prompting

Para un asistente sobre normativa universitaria interna (documentos privados que cambian cada semestre): (1) ¿Cuándo elegirías RAG? (2) ¿Cuándo elegirías fine-tuning? (3) ¿Qué papel juega el prompt engineering en ambos? Da una respuesta razonada.

### (1) ¿Cuándo RAG?

RAG es preferible cuando el conocimiento es **privado y cambia con frecuencia** y necesitamos **trazabilidad** (saber de qué documento viene la respuesta). En este caso, la normativa cambia cada semestre, por lo que RAG permite actualizar el corpus sin reentrenar el modelo.

### (2) ¿Cuándo Fine-tuning?

Fine-tuning solo si el estilo o la tarea es **estable** y hay datos de alta calidad para entrenar (y no cambia cada semestre). Por ejemplo, si se quisiera que el modelo adoptase un tono formal específico de la universidad de forma permanente.

### (3) Papel del Prompt Engineering

Prompting es **complementario** a ambos enfoques: define el formato de respuesta, fuerza la inclusión de citas, controla la abstención cuando no hay información suficiente y reduce alucinaciones.

## Ejercicio A3 - Evaluación de RAG

Propón un plan de evaluación con al menos 3 métricas o criterios: (i) métricas de retrieval (p.ej., Recall@k), (ii) métricas de respuesta (p.ej., exactitud/fidelidad), (iii) métricas de robustez (p.ej., sensibilidad a perturbaciones). Indica cómo obtendrías datos de evaluación (conjunto de preguntas, gold answers, etc.).

### Plan de evaluación

| Categoría | Métricas | Descripción |
|---|---|---|
| **(i) Retrieval** | Recall@k, MRR | Medir sobre preguntas con documentos gold si los chunks relevantes aparecen en el top-k recuperado. |
| **(ii) Respuesta** | Exactitud, Faithfulness, EM/F1 | Evaluar si la respuesta es correcta y si está **apoyada en el contexto** recuperado (no alucinada). EM/F1 si hay respuestas canónicas. |
| **(iii) Robustez** | Sensibilidad a perturbaciones | Perturbaciones de la pregunta (paráfrasis), cambios de k, introducción de ruido en el corpus. |

### Obtención de datos de evaluación

- Crear un banco de **30–50 preguntas**.
- Para cada pregunta: documento(s) relevante(s) y respuesta esperada.
- **Evaluación humana** en casos ambiguos donde las métricas automáticas no sean suficientes.